This notebook is trying to go through the GPS localization algorithms in Global Position System: Theory and Application Volume 1 with Paul Zarchan as Editor-in-Chief. Navigation algorithms are mentioned in Chapter 9 written by Axelrad and Brown

# Point Solution Example

Described in Chapter 9, III B

In [1]:
import numpy as np
import scipy.constants as const
import matplotlib.pyplot as plt

In [8]:
# known satellite locations
satellitePositions = np.array([
        [22808160.9, -12005866.6, -6609526.5],
        [21141179.5, -2355056.3, -15985716.1],
        [20438959.3, -4238967.1, 16502090.2],
        [18432296.2, -18613382.5, -4672400.8],
        [21772117.8, 13773269.7, 6656636.4],
        [15561523.9, 3469098.6, -21303596.2],
        [13773316.6, 15929331.4, -16266254.4]
    ])
nSatellites = satellitePositions.shape[0]
print("number of satellites: ", nSatellites)

# ground truth
xTrue = np.array([6378137.0, 0.0, 0.0, 85000.0])

measuredPseudorange = np.array([
        21480623.2,
        21971919.2,
        22175603.9,
        22747561.5,
        21787252.3,
        23541613.4,
        24022907.4
    ])

# a priori estimate of state in the form of [x, y, z, clock bias]
xHat = np.array([6377000.0, 3000.0, 4000.0, 0.0])

# # the position difference vector pointing from a priori estimate to satellite positions
# positionDiff = satellitePositions - xHat[0:3]
# normalizedPos = np.linalg.norm(positionDiff, axis = 1)
# # pseudorange also considers the distance equivalent of time offset
# computedPseudorange = normalizedPos + xHat[3]
# print("computed pseudorange:\n", np.round(computedPseudorange[:,None], 1))
# print()

# print("pseudorange residual:\n", (computedPseudorange - measuredPseudorange)[:,None])
# print()

# # line of sight vectors are just the normalized version of the position difference vectors
# lineOfSight = np.divide(positionDiff, normalizedPos[:,None])
# print("line of sight vectors:\n", lineOfSight)
# print()

# # G = [-line of sight vectors which represent geometry, frequency bias(?)]
# G = np.concatenate((-lineOfSight, np.ones((nSatellites,1))), axis=1)
# print("G:\n", G)

def stateUpdate(Xprev, measuredPseudorange, almanac, **args):
    print("#debug: ", Xprev)
    nSatellites = almanac.shape[0]
    positionDifference = almanac - Xprev[0:3]
    normalizedPositionDifference = np.linalg.norm(positionDifference, axis = 1)
    computedPseudorange = normalizedPositionDifference + Xprev[3]
    pseudorangeResidual = computedPseudorange - measuredPseudorange
    lineOfSight = np.divide(positionDifference, normalizedPositionDifference[:,None])
    G = np.concatenate((-lineOfSight, np.ones((nSatellites, 1))), axis = 1)
    GTG = np.dot(G.T, G)
    GTGinv = np.linalg.inv(GTG)
    A = np.dot(GTGinv, G.T)
    deltaX = np.dot(A, pseudorangeResidual)
    Xnew = Xprev - deltaX
    
    if 'errorEstimate' in args and args['errorEstimate'] != 'none':
        # error measurements are required
        if args['errorEstimate'] == 'covariance':
            # we want to provide error estimate as a covariance matrix    
            try:
                sigma = args['measurementSigma']
            except AttributeError:
                raise AttributeError("couldnt find measurementSigma in input to stateUpdate function")
            R = np.diag(args['measurementSigma'])
            B = np.dot(G, GTGinv)
            E = np.dot(np.dot(A, R), B)
            return (Xnew, E)
        elif args['errorEstimate'] == 'DOP':
            DOP = GTGinv.diagonal()
            return (Xnew, DOP)
    else:
        # only provide new location 
        return Xnew

# iterate and observe evolution
i = 0
while i <= 5:
    i = i + 1
    if i == 1:
        xPrev = xHat
    else:
        xPrev = xUpdated
    (xUpdated, DOP) = stateUpdate(xPrev, measuredPseudorange, satellitePositions, errorEstimate = 'DOP')
    print("iteration {}".format(i))
    print("updated position = ({:.2f}, {:.2f}, {:.2f})".format(xUpdated[0], xUpdated[1], xUpdated[2]))
    print("updated clock bias = {:f}".format(xUpdated[3]))
    err = xUpdated - xTrue
    print("residual position = ({:.2f}, {:.2f}, {:.2f})".format(err[0], err[1], err[2]))
    print("residual clock = {:.2f}".format(err[3]))
    print("sqrt DOP elements:")
    print(np.sqrt(DOP))
    print()

number of satellites:  7
#debug:  [6.377e+06 3.000e+03 4.000e+03 0.000e+00]
iteration 1
updated position = (6378131.72, 3.23, 6.86)
updated clock bias = 84996.327344
residual position = (-5.28, 3.23, 6.86)
residual clock = -3.67
sqrt DOP elements:
[2.99339657 0.78837158 0.794988   1.86006872]

#debug:  [6.37813172e+06 3.23349614e+00 6.85868781e+00 8.49963273e+04]
iteration 2
updated position = (6378131.41, 3.37, 7.04)
updated clock bias = 84995.731981
residual position = (-5.59, 3.37, 7.04)
residual clock = -4.27
sqrt DOP elements:
[2.99302019 0.78842389 0.79476191 1.8597075 ]

#debug:  [6.37813141e+06 3.37045835e+00 7.04402240e+00 8.49957320e+04]
iteration 3
updated position = (6378131.41, 3.37, 7.04)
updated clock bias = 84995.731981
residual position = (-5.59, 3.37, 7.04)
residual clock = -4.27
sqrt DOP elements:
[2.99302024 0.78842389 0.79476193 1.85970756]

#debug:  [6.37813141e+06 3.37045842e+00 7.04402235e+00 8.49957320e+04]
iteration 4
updated position = (6378131.41, 3.37, 7.04